In [6]:
import gym
import numpy as np
from math import sqrt
import time
import random

# Globals

In [12]:

# (state,action) : [value,count]
state_action = {}
env = gym.make('CartPole-v1')
scale = 30
exploration = 0.9

In [13]:
def get_discrete_obs(observation):
    t = (observation* scale).astype(int)
    return tuple(t)

def get_action(observation):
    t = random.random()
#     print(exploration)
    if t < exploration:
        return env.action_space.sample()
    else:
        action = env.action_space.sample()
        best_score = 0
        for k,v in state_action.items():
            if k[0] == observation and v[0] > best_score:
                action = k[1]
                best_score = v[0]
        return action
            
    

def get_episode():
    ## (state,action,reward)
    episode = []
    observation = env.reset()
    observation = get_discrete_obs(observation)
    
    while True:
        action = get_action(observation)
        observation, reward, done, _ = env.step(action)
        observation = get_discrete_obs(observation)
        episode.append([observation,action,reward])
        if done:
            break
    return episode

def eval_episode(episode):
    episode.reverse()
    reward_so_far = 0
    
    for state,action,reward in episode:
#         print(state,action,reward)
        reward_so_far = reward_so_far + reward
        
        if (state,action) in state_action:
            value,count = state_action[(state,action)]
            new_val = (value * count + reward_so_far) / (count + 1)
            state_action[(state,action)] = [new_val,count + 1]
        
        else:
            state_action[(state,action)] = [reward_so_far,1]
            

            
    

In [14]:

def train():
    global exploration
    avg_len = 0
    max_len = 1000
    for i in range(1,10000):
        ep = get_episode()
        eval_episode(ep)
        avg_len = avg_len + len(ep)
        
        if i%max_len == 0:
            print("Avg is",avg_len/max_len)
            print("Exploring",exploration)
            avg_len = 0
        exploration = exploration * 0.9999
        


In [15]:
train()

Avg is 21.881
Exploring 0.8144310473074298
Avg is 20.963
Exploring 0.7369240011391069
Avg is 20.186
Exploring 0.6667930762834472
Avg is 20.05
Exploring 0.6033363086183618
Avg is 19.693
Exploring 0.5459185379160898
Avg is 19.381
Exploring 0.4939650503098715


KeyboardInterrupt: 